In [1]:
import Pkg

Pkg.add("SparseArrays")
Pkg.add("LinearAlgebra")
Pkg.add("SpecialFunctions")
Pkg.add("Arpack")   
Pkg.add("DelimitedFiles")
Pkg.add("NPZ")
Pkg.add("Random")
Pkg.add("Statistics")
Pkg.add("ExpmV")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.11/Project.toml`
  No Changes to `~/.julia/environments/v1.11/Manifest.toml`
   Resolving package version

In [2]:
using SparseArrays
using LinearAlgebra
using SpecialFunctions  # for besselj
using Arpack  # for eigs on sparse matrices
using Statistics
using Random
using DelimitedFiles
using NPZ
using ExpmV


In [3]:
function von_neumann_entropy(ψ::Vector{<:Complex}, cut::Int, L::Int)
    dimA = 3^cut
    dimB = 3^(L - cut)
    ψ_matrix = reshape(ψ, (dimA, dimB))
    svals = svdvals(ψ_matrix)
    S = 0.0
    for s in svals
        p = abs2(s)
        if p > 0 && p > 1e-15
            S -= p * log(p)
        end
    end
    return S
end

von_neumann_entropy (generic function with 1 method)

In [4]:
using LinearAlgebra
using Random

function random_product_state(L::Int)
    product_state = nothing

    for i in 1:L
        θ1 = rand() * π
        θ2 = rand() * π
        ϕ1 = rand() * 2π
        ϕ2 = rand() * 2π

        c1 = cos(θ1 / 2)
        c2 = exp(im * ϕ1) * sin(θ1 / 2) * sin(θ2 / 2)
        c3 = exp(im * ϕ2) * sin(θ1 / 2) * cos(θ2 / 2)

        temp_state = [c1, c2, c3]

        if i == 1
            product_state = temp_state
        else
            product_state = kron(product_state, temp_state)
        end
    end

    # Normalize the state
    product_state /= norm(product_state)

    return product_state
end

random_product_state (generic function with 1 method)

In [5]:
function Hamiltonian(L)
    # Define Pauli matrices as complex sparse matrices
    id = sparse(ComplexF64[1 0 0; 0 1 0; 0 0 1])
    sx = 1/sqrt(2) * sparse(ComplexF64[0 1 0; 1 0 1; 0 1 0])
    sy = 1/sqrt(2) * sparse(ComplexF64[0 -im 0; im 0 -im; 0 im 0])
    sz = sparse(ComplexF64[1 0 0; 0 0 0; 0 0 -1])

    sp = 1/sqrt(2) * (sx + im * sy)
    sm = 1/sqrt(2) * (sx - im * sy)

    # Preallocate vectors of operators with correct type
    #sx_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    #sy_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    szl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz operators
    sz2l = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz² operators
    spl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sp operators
    sml = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sm operators

    for i_site in 1:L
        p_ops = fill(id, L)
        m_ops = fill(id, L)
        z_ops = fill(id, L)
        z2_ops = fill(id, L)
        p_ops[i_site] = sp
        m_ops[i_site] = sm
        z_ops[i_site] = sz
        z2_ops[i_site] = sz^2

        # Build the full operator by tensoring
        P = p_ops[1]
        M = m_ops[1]
        Z = z_ops[1]
        Z2 = z2_ops[1]
        for j in 2:L
            P = kron(P, p_ops[j])
            M = kron(M, m_ops[j])
            Z = kron(Z, z_ops[j])
            Z2 = kron(Z2, z2_ops[j])
        end

        spl[i_site] = P
        sml[i_site] = M
        szl[i_site] = Z
        sz2l[i_site] = Z2
    end

    dim = 3^L
    H = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        H += ((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2) + adjoint((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2)))
    end

    return H, sz2l, szl 
end

Hamiltonian (generic function with 1 method)

In [6]:
using ExpmV

function time_evolution(ψ::Vector{ComplexF64}, dt::Float64, L)
    ψ /= norm(ψ)
    H, _, _ = Hamiltonian(L)

    # Apply exp(-im * H * dt) directly to ψ
    ψ_new = expmv(-im * dt, H, ψ)

    # Normalize the state
    ψ_new /= norm(ψ_new)
    
    return ψ_new
end


time_evolution (generic function with 1 method)

In [ ]:
state = time_evolution(random_product_state(13), 1.0, 13)

In [ ]:
von_neumann_entropy(state, 6, 13)

In [ ]:
function Entropy_t(L::Int, T::Float64, dt::Float64, p::Float64, direction::String, shot::Int)
    Random.seed!(shot)  # Set random seed
    s_t = random_product_state(L)
    S_list = Float64[]

    # Define Hamiltonian and local observables
    H, sz2l, szl = Hamiltonian(L)
    
    if direction == "Z2"
        sm_list = sz2l
    elseif direction == "Z"
        sm_list = szl
    else
        error("Invalid direction: $direction. Choose \"Z2\" or \"Z\".")
    end


    steps = Int(floor(T / dt))

    for _ in 1:steps
        push!(S_list, von_neumann_entropy(s_t, L÷2, L))
        #push!(S_list, I3(s_t))

        # Time evolution
        s_t = time_evolution(s_t, dt)
        s_t /= norm(s_t)

        # Measurements
        if p != 0
            for l in 1:L
                x = rand()
                if x < p
                    m_op = sm_list[l]
                    p_m = real(s_t' * (m_op * s_t))
                    if rand() < p_m
                        s_t = (m_op * s_t) / sqrt(p_m)
                    else
                        s_t = (s_t - m_op * s_t) / sqrt(1 - p_m)
                    end
                end
            end
        end
    end

    # Save result to disk
    filename = "L$(L),T$(T),dt$(dt),p$(p),dir$(direction),s$(shot).npy"
    #npzwrite(filename, S_list)

    return S_list
end

In [ ]:
Entropy_t(8, 100.0, 1.0, 1.0, "Z2", 1)

In [ ]:
function I3(psi::Vector{ComplexF64})
    log3(x) = log(x) / log(3)

    L = Int(round(log3(length(psi))))
    qL = L ÷ 4

    SA   = von_neumann_entropy(psi, qL, L)
    SB   = von_neumann_entropy(psi, 2qL, L)
    SC   = von_neumann_entropy(psi, 3qL, L)
    SABC = von_neumann_entropy(psi, L, L)
    SAB  = von_neumann_entropy(psi, 2qL, L)
    SAC  = von_neumann_entropy(psi, 3qL, L)
    SBC  = von_neumann_entropy(psi, 3qL, L)
    


    return SA + SB + SC + SABC - SAB - SAC - SBC
end

In [ ]:
function Entropy_t_z(L::Int, T::Float64, dt::Float64, p::Float64, shot::Int)
    Random.seed!(shot)  # Set random seed
    s_t = random_product_state(L)
    S_list = Float64[]

    # Define Hamiltonian and local observables
    H, sz2l, szl = Hamiltonian(L)
    sm_list = szl
    steps = Int(floor(T / dt))

    for _ in 1:steps
        push!(S_list, von_neumann_entropy(s_t, L÷2, L))
        #push!(S_list, I3(s_t))

        # Time evolution
        s_t = time_evolution(s_t, dt)
        s_t /= norm(s_t)

        # Measurements
        if p != 0
            for l in 1:L
                x = rand()
                if x < p
                    m_op = sm_list[l]
                    p_m_mone = 0.5 * real(s_t' * (sz2l[l]-sm_list[l]) * s_t)
                    p_m_one = 0.5 * real(s_t' * (sz2l[l] + sm_list[l]) * s_t)
                    x1 = rand()
                    if x1 < p_m_mone
                        s_t = 0.5 * ((sz2l[l]-sm_list[l]) * s_t) / sqrt(p_m_mone)
                    elseif p_m_mone < x1 < p_m_one + p_m_mone
                        s_t = 0.5 * ((sz2l[l]+sm_list[l]) * s_t) / sqrt(p_m_one)
                    else
                        s_t = (s_t - sz2l[l] * s_t) / sqrt(1 - p_m_mone - p_m_one)
                    end
                end
            end
        end
    end

    # Save result to disk
    filename = "L$(L),T$(T),dt$(dt),p$(p),dirZ,s$(shot).npy"
    npzwrite(filename, S_list)

    return S_list
end

In [ ]:
include("s3.jl")

In [ ]:
Entropy_t_z(8, 100.0, 1.0, 0.3, 398)

for L in [8]
    for T in [100.0]
        for dt in [1.0]
            for p in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]
                for shot in 1:500
                    Entropy_t_z(L, T, dt, p, shot)
                end
            end
        end
    end
end

This took 320 minutes to run

### Ancilla purification code

In [ ]:
using SparseArrays
using LinearAlgebra
using Arpack
using Statistics
using Random
using DelimitedFiles
using NPZ
using ExpmV

function von_neumann_entropy(ψ::Vector{<:Complex}, cut::Int, L::Int)
    dimA = 3^cut
    dimB = 3^(L - cut)
    ψ_matrix = reshape(ψ, (dimA, dimB))
    svals = svdvals(ψ_matrix)
    S = 0.0
    for s in svals
        if s > 1e-15
            p = abs2(s)
            S -= p * log(p)
        end
    end
    return S
end

function random_product_state(L::Int)
    product_state = nothing

    for i in 1:L
        θ1 = rand() * π
        θ2 = rand() * π
        ϕ1 = rand() * 2π
        ϕ2 = rand() * 2π

        c1 = cos(θ1 / 2)
        c2 = exp(im * ϕ1) * sin(θ1 / 2) * sin(θ2 / 2)
        c3 = exp(im * ϕ2) * sin(θ1 / 2) * cos(θ2 / 2)

        temp_state = [c1, c2, c3]

        if i == 1
            product_state = temp_state
        else
            product_state = kron(product_state, temp_state)
        end
    end

    # Normalize the state
    product_state /= norm(product_state)

    return product_state
end

function time_evolution(ψ::Vector{ComplexF64}, dt::Float64, L)
    ψ /= norm(ψ)
    Ha, Hb, Hc, _, _ = Hamiltonian(L)
    a = 2π * rand()
    b = 2π * rand()
    c = 2π * rand()
    # Apply exp(-im * H * dt) directly to ψ
    H = a * Ha + b * Hb + c * Hc
    ψ_new = expmv(-im * dt, H, ψ)

    # Normalize the state
    ψ_new /= norm(ψ_new)
    
    return ψ_new
end

function I3(psi::Vector{ComplexF64})
    log3(x) = log(x) / log(3)

    L = Int(round(log3(length(psi))))
    qL = L ÷ 4

    SA   = von_neumann_entropy(psi, qL, L)
    SB   = von_neumann_entropy(psi, 2qL, L)
    SC   = von_neumann_entropy(psi, 3qL, L)
    SABC = von_neumann_entropy(psi, L, L)
    SAB  = von_neumann_entropy(psi, 2qL, L)
    SAC  = von_neumann_entropy(psi, 3qL, L)
    SBC  = von_neumann_entropy(psi, 3qL, L)
    
    return SA + SB + SC + SABC - SAB - SAC - SBC
end

### This function is for Z measurements only

function Entropy_t_z(L::Int, T::Float64, dt::Float64, p::Float64, shot::Int)
    Random.seed!(shot)  # Set random seed
    s_t = random_product_state(L)
    S_list = Float64[]

    # Define Hamiltonian and local observables
    _, _, _, sz2l, szl = Hamiltonian(L)
    sm_list = szl
    steps = Int(floor(T / dt))

    for _ in 1:steps
        #push!(S_list, von_neumann_entropy(s_t, L÷2, L))
        push!(S_list, I3(s_t))

        # Time evolution
        s_t = time_evolution(s_t, dt, L)
        s_t /= norm(s_t)

        # Measurements
        if p != 0
            for l in 1:L
                x = rand()
                if x < p
                    m_op = sm_list[l]
                    p_m_mone = 0.5 * real(s_t' * (sz2l[l]-sm_list[l]) * s_t)
                    p_m_one = 0.5 * real(s_t' * (sz2l[l] + sm_list[l]) * s_t)
                    x1 = rand()
                    if x1 < p_m_mone
                        s_t = 0.5 * ((sz2l[l]-sm_list[l]) * s_t) / sqrt(p_m_mone)
                    elseif p_m_mone < x1 < p_m_one + p_m_mone
                        s_t = 0.5 * ((sz2l[l]+sm_list[l]) * s_t) / sqrt(p_m_one)
                    else
                        s_t = (s_t - sz2l[l] * s_t) / sqrt(1 - p_m_mone - p_m_one)
                    end
                end
            end
        end
    end

    # Save result to disk
    filename = "L$(L),T$(T),dt$(dt),p$(p),dirZ,s$(shot).npy"
    npzwrite(filename, S_list)

    return S_list
end

function Hamiltonian(L)
    # Define Pauli matrices as complex sparse matrices
    id = sparse(ComplexF64[1 0 0; 0 1 0; 0 0 1])
    sx = 1/sqrt(2) * sparse(ComplexF64[0 1 0; 1 0 1; 0 1 0])
    sy = 1/sqrt(2) * sparse(ComplexF64[0 -im 0; im 0 -im; 0 im 0])
    sz = sparse(ComplexF64[1 0 0; 0 0 0; 0 0 -1])

    sp = 1/sqrt(2) * (sx + im * sy)
    sm = 1/sqrt(2) * (sx - im * sy)

    # Preallocate vectors of operators with correct type
    #sx_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    #sy_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    szl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz operators
    sz2l = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz² operators
    spl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sp operators
    sml = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sm operators

    for i_site in 1:L
        p_ops = fill(id, L)
        m_ops = fill(id, L)
        z_ops = fill(id, L)
        z2_ops = fill(id, L)
        p_ops[i_site] = sp
        m_ops[i_site] = sm
        z_ops[i_site] = sz
        z2_ops[i_site] = sz^2

        # Build the full operator by tensoring
        P = p_ops[1]
        M = m_ops[1]
        Z = z_ops[1]
        Z2 = z2_ops[1]
        for j in 2:L
            P = kron(P, p_ops[j])
            M = kron(M, m_ops[j])
            Z = kron(Z, z_ops[j])
            Z2 = kron(Z2, z2_ops[j])
        end

        spl[i_site] = P
        sml[i_site] = M
        szl[i_site] = Z
        sz2l[i_site] = Z2
    end

    dim = 3^L
    Ha = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Ha += ((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2) + adjoint((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2)))
    end

    Hb = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Hb += (spl[i] * szl[i] * sml[ip] * szl[ip] + sml[i] * szl[i] * spl[ip] * szl[ip] + adjoint(spl[i] * szl[i] * sml[ip] * szl[ip]) + adjoint(sml[i] * szl[i] * spl[ip] * szl[ip]))
    end

    Hc = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Hc += (spl[i]^2 * spl[ip] * szl[ip] + sml[i]^2 * sml[ip] * szl[ip] + adjoint(spl[i]^2 * spl[ip] * szl[ip]) + adjoint(sml[i]^2 * sml[ip] * szl[ip]))
    end

    return Ha, Hb, Hc, sz2l, szl 
end

In [ ]:
function Hamiltonian(L)
    # Define Pauli matrices as complex sparse matrices
    id = sparse(ComplexF64[1 0 0; 0 1 0; 0 0 1])
    sx = 1/sqrt(2) * sparse(ComplexF64[0 1 0; 1 0 1; 0 1 0])
    sy = 1/sqrt(2) * sparse(ComplexF64[0 -im 0; im 0 -im; 0 im 0])
    sz = sparse(ComplexF64[1 0 0; 0 0 0; 0 0 -1])

    sp = 1/sqrt(2) * (sx + im * sy)
    sm = 1/sqrt(2) * (sx - im * sy)

    # Preallocate vectors of operators with correct type
    #sx_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    #sy_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    szl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz operators
    sz2l = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz² operators
    spl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sp operators
    sml = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sm operators

    for i_site in 1:L
        p_ops = fill(id, L)
        m_ops = fill(id, L)
        z_ops = fill(id, L)
        z2_ops = fill(id, L)
        p_ops[i_site] = sp
        m_ops[i_site] = sm
        z_ops[i_site] = sz
        z2_ops[i_site] = sz^2

        # Build the full operator by tensoring
        P = p_ops[1]
        M = m_ops[1]
        Z = z_ops[1]
        Z2 = z2_ops[1]
        for j in 2:L
            P = kron(P, p_ops[j])
            M = kron(M, m_ops[j])
            Z = kron(Z, z_ops[j])
            Z2 = kron(Z2, z2_ops[j])
        end

        spl[i_site] = P
        sml[i_site] = M
        szl[i_site] = Z
        sz2l[i_site] = Z2
    end

    dim = 3^L
    Ha = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Ha += ((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2) + adjoint((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2)))
    end

    Hb = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Hb += (spl[i] * szl[i] * sml[ip] * szl[ip] + sml[i] * szl[i] * spl[ip] * szl[ip] + adjoint(spl[i] * szl[i] * sml[ip] * szl[ip]) + adjoint(sml[i] * szl[i] * spl[ip] * szl[ip]))
    end

    Hc = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Hc += (spl[i]^2 * spl[ip] * szl[ip] + sml[i]^2 * sml[ip] * szl[ip] + adjoint(spl[i]^2 * spl[ip] * szl[ip]) + adjoint(sml[i]^2 * sml[ip] * szl[ip]))
    end

    return Ha, Hb, Hc, sz2l, szl 
end

In [ ]:
function entropy_vn(ψ::Vector{<:Complex}, L::Int, subsystem::AbstractArray{Int})
    cut = length(subsystem)
    dimA = 3^cut
    dimB = 3^(L - cut)
    ψ_matrix = reshape(ψ, (dimA, dimB))
    svals = svdvals(ψ_matrix)
    S = 0.0
    for s in svals
        if s > 1e-15
            p = abs2(s)
            S -= p * log(p)
        end
    end
    return S
end

Code run

In [ ]:
include("s3.jl")

In [ ]:
@time Entropy_t_z(8, 100.0, 1.0, 0.4, 3980)

### ITensors benchmarking

In [ ]:
function Hamiltonian(L)
    # Define Pauli matrices as complex sparse matrices
    id = sparse(ComplexF64[1 0 0; 0 1 0; 0 0 1])
    sx = 1/sqrt(2) * sparse(ComplexF64[0 1 0; 1 0 1; 0 1 0])
    sy = 1/sqrt(2) * sparse(ComplexF64[0 -im 0; im 0 -im; 0 im 0])
    sz = sparse(ComplexF64[1 0 0; 0 0 0; 0 0 -1])

    sp = 1/sqrt(2) * (sx + im * sy)
    sm = 1/sqrt(2) * (sx - im * sy)

    # Preallocate vectors of operators with correct type
    #sx_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    #sy_list = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L)
    szl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz operators
    sz2l = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sz² operators
    spl = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sp operators
    sml = Vector{SparseMatrixCSC{ComplexF64, Int}}(undef, L) # List of sm operators

    for i_site in 1:L
        p_ops = fill(id, L)
        m_ops = fill(id, L)
        z_ops = fill(id, L)
        z2_ops = fill(id, L)
        p_ops[i_site] = sp
        m_ops[i_site] = sm
        z_ops[i_site] = sz
        z2_ops[i_site] = sz^2

        # Build the full operator by tensoring
        P = p_ops[1]
        M = m_ops[1]
        Z = z_ops[1]
        Z2 = z2_ops[1]
        for j in 2:L
            P = kron(P, p_ops[j])
            M = kron(M, m_ops[j])
            Z = kron(Z, z_ops[j])
            Z2 = kron(Z2, z2_ops[j])
        end

        spl[i_site] = P
        sml[i_site] = M
        szl[i_site] = Z
        sz2l[i_site] = Z2
    end

    dim = 3^L
    Ha = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Ha += ((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2) + adjoint((spl[i]^2 * sml[ip]^2 + sml[i]^2 * spl[ip]^2)))
    end

    Hb = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Hb += (spl[i] * szl[i] * sml[ip] * szl[ip] + sml[i] * szl[i] * spl[ip] * szl[ip] + adjoint(spl[i] * szl[i] * sml[ip] * szl[ip]) + adjoint(sml[i] * szl[i] * spl[ip] * szl[ip]))
    end

    Hc = spzeros(ComplexF64, dim, dim)

    for i in 1:L
        ip = (i % L) + 1  # Periodic boundary
        Hc += (spl[i]^2 * spl[ip] * szl[ip] + sml[i]^2 * sml[ip] * szl[ip] + adjoint(spl[i]^2 * spl[ip] * szl[ip]) + adjoint(sml[i]^2 * sml[ip] * szl[ip]))
    end

    return Ha
end

In [ ]:
Ha = Hamiltonian(8)

In [ ]:
## Check if Ha is Hermitian
ishermitian(Ha)

In [ ]:
using LinearAlgebra
using SparseArrays
using Arpack: eigs

In [ ]:
vals, vecs = eigs(Ha; nev=1, which=:SR)
lowest_eig = vals[1]

println("Lowest eigenvalue: ", lowest_eig)

In [ ]:
@show op("S+", s)

In [ ]:
id = [1 0 0; 0 1 0; 0 0 1];
sx = 1/sqrt(2) * [0 1 0; 1 0 1; 0 1 0];
sy = 1/sqrt(2) * [0 -im 0; im 0 -im; 0 im 0];

In [ ]:
sp = 1/sqrt(2) * (sx + im * sy)

In [ ]:
sm = 1/sqrt(2) * (sx - im * sy)

In [ ]:
sp' == sm

In [ ]:
using ITensorMPS

N = 8  #number of sites
sites = siteinds("S=1",N; conserve_qns=true) #spin 1 in each sites

  os = OpSum()
  for j=1:N
    os += 1.0 ,"S+ * S+",j,"S- * S-",mod(j,N)+1   #I think this is the Ham in Sriram's paper
    os += 1.0 ,"S- * S-",j,"S+ * S+",mod(j,N)+1
    os += 1.0 ,"S+ * S+",mod(j,N)+1,"S- * S-",j   # H.c. terms
    os += 1.0 ,"S- * S-",mod(j,N)+1,"S+ * S+",j
    #os += 1.0 ,"S+ * S+",j+1,"S- * S-",j
    #os += 1.0 , "S- * S-", j+1, "S+ * S+",j
  end
  H = MPO(os,sites)

  nsweeps = 15 # number of sweeps is 15
  maxdim = [10,20,100,100,200,200,400,400,500] # gradually increase states kept
  cutoff = [1E-10] # desired truncation error
  
  states = [isodd(n) ? "Up" : "Dn" for n in 1:N] #Neel state-- we know GS is in S^z=0 sector
  psi0 = MPS(sites, states) #constructing the state as MPS which is taken as variational ansatz in next step

  @time energy,psigs = dmrg(H,psi0;nsweeps,maxdim,cutoff);

In [ ]:
using ITensorMPS

N = 8 #number of sites
sites = siteinds("S=1",N; conserve_qns=true) #spin 1 in each sites

  os = OpSum()
  for j=1:N
    os += 1.0 ,"S+ * S+",j,"S- * S-",mod(j,N)+1   #I think this is the Ham in Sriram's paper
    os += 1.0 ,"S- * S-",j,"S+ * S+",mod(j,N)+1
    os += 1.0 ,"S+ * S+",j+1,"S- * S-",j
    os += 1.0 , "S- * S-", j+1, "S+ * S+",j
  end
  H = MPO(os,sites)

  nsweeps = 15 # number of sweeps is 15
  maxdim = [10,20,100,100,200,200,400,400,500] # gradually increase states kept
  cutoff = [1E-10] # desired truncation error
  
  states = [isodd(n) ? "Up" : "Dn" for n in 1:N] #Neel state-- we know GS is in S^z=0 sector
  psi0 = MPS(sites, states) #constructing the state as MPS which is taken as variational ansatz in next step

  energy,psigs = dmrg(H,psi0;nsweeps,maxdim,cutoff);

In [ ]:
using ITensors, ITensorMPS, Random, HDF5, StatsBase
#using Plots

N = 8
sites = siteinds("S=1", N; conserve_qns=true)

os = OpSum()
for j = 1:N ####### change to N-1 if you're doing OBC
  os += 1.0, "S+ * S+", j, "S- * S-", mod(j, N) + 1
  os += 1.0, "S- * S-", j, "S+ * S+", mod(j, N) + 1
  os += 1.0 ,"S+ * S+",mod(j,N)+1,"S- * S-",j   # H.c. terms
  os += 1.0 ,"S- * S-",mod(j,N)+1,"S+ * S+",j
end
H = MPO(os, sites)

# ----- initial product state 
state = [isodd(j) ? "Up" : "Dn" for j in 1:N]       # Néel: isodd(j) ? "Up" : "Dn"
psi = productMPS(sites, state)

#func to compute EE at any site
function entropy_von_neumann(psi, b)
  s = siteinds(psi)
  orthogonalize!(psi, b)
  if b == 1
    U, S, V = svd(psi[b], (siteind(psi, b),))
  else
    U, S, V = svd(psi[b], (linkind(psi, b - 1), siteind(psi, b)))
  end
  SvN = 0.0
  for n in 1:dim(S, 1)
    p = S[n, n]^2
    if p > 0
      SvN -= p * log(p)
    end
  end
  return SvN
end

# ----- TDVP evolution + half-chain entropy at every step -----
tstep = 0.02
tend = 5.0
nsw = round(Int, tend / tstep)
tdvp_maxdim = 500

Time = Float64[]
SvN_half = Float64[]

push!(Time, 0.0)
push!(SvN_half, entropy_von_neumann(psi, N ÷ 2))

for n in 1:nsw
  ncenter = 1
  if maxlinkdim(psi) < tdvp_maxdim
    ncenter = 2
  end

  psi = tdvp(H, -im * tstep, psi;
             time_step = -im * tstep,
             reverse_step = true,
             normalize = false,
             cutoff = 0.0,
             maxdim = tdvp_maxdim,
             nsite = ncenter,
             updater_backend = "exponentiate",
             outputlevel = 1)

  tnow = n * tstep
  push!(Time, tnow)
  push!(SvN_half, entropy_von_neumann(psi, N ÷ 2))

  GC.gc(true)
end

# ----- save results -----
# fname = "SvN_half_vs_time_S1_N$(N)_dt$(tstep)_T$(tend).h5"
# h5write(fname, "time", Time)
# h5write(fname, "SvN_half", SvN_half)
# println("Saved: ", fname)


#plot(Time,SvN_half)
#savefig("SvN_half_vs_time_S1_N$(N)_dt$(tstep)_T$(tend)_tdvp.png")

In [1]:
1+1

2